## PROYECTO FINAL DE BIG DATA PROCESSING

Instalación de Spark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q http://apache.osuosl.org/spark/spark-3.3.3/spark-3.3.3-bin-hadoop3.tgz

In [ ]:
!tar xf spark-3.3.3-bin-hadoop3.tgz

In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.3-bin-hadoop3"

In [ ]:
!pip install -q findspark

In [ ]:
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Proyecto Final Processing').getOrCreate()

Comprobación de instalación

In [ ]:
spark

## Creando el dataframe a partir del data

In [ ]:
df21 = spark.read.option('header','true').csv('./world-happiness-report-2021.csv')


In [ ]:
df21.show()

In [ ]:
df21.printSchema()

In [ ]:
df21 = df21.withColumnRenamed('Country name','Country_name').withColumnRenamed('Ladder score','Ladder_score').withColumnRenamed('Regional indicator','Regional_indicator')
df21.select('Country_name','Ladder_score','Standard error of ladder score').show()

+--------------+------------+------------------------------+
|  Country_name|Ladder_score|Standard error of ladder score|
+--------------+------------+------------------------------+
|       Finland|       7.842|                         0.032|
|       Denmark|       7.620|                         0.035|
|   Switzerland|       7.571|                         0.036|
|       Iceland|       7.554|                         0.059|
|   Netherlands|       7.464|                         0.027|
|        Norway|       7.392|                         0.035|
|        Sweden|       7.363|                         0.036|
|    Luxembourg|       7.324|                         0.037|
|   New Zealand|       7.277|                         0.040|
|       Austria|       7.268|                         0.036|
|     Australia|       7.183|                         0.041|
|        Israel|       7.157|                         0.034|
|       Germany|       7.155|                         0.040|
|        Canada|       7

In [ ]:
from pyspark.sql.functions import min, max, col


Transformando el índice de felicidad de string a flotante

In [ ]:
from pyspark.sql.types import IntegerType
df21 = df21.withColumn('Ladder_score', col('Ladder_score').cast('float'))
df21.printSchema()


root
 |-- Country_name: string (nullable = true)
 |-- Regional_indicator: string (nullable = true)
 |-- Ladder_score: float (nullable = true)
 |-- Standard error of ladder score: string (nullable = true)
 |-- upperwhisker: string (nullable = true)
 |-- lowerwhisker: string (nullable = true)
 |-- Logged GDP per capita: string (nullable = true)
 |-- Social support: string (nullable = true)
 |-- Healthy life expectancy: string (nullable = true)
 |-- Freedom to make life choices: string (nullable = true)
 |-- Generosity: string (nullable = true)
 |-- Perceptions of corruption: string (nullable = true)
 |-- Ladder score in Dystopia: string (nullable = true)
 |-- Explained by: Log GDP per capita: string (nullable = true)
 |-- Explained by: Social support: string (nullable = true)
 |-- Explained by: Healthy life expectancy: string (nullable = true)
 |-- Explained by: Freedom to make life choices: string (nullable = true)
 |-- Explained by: Generosity: string (nullable = true)
 |-- Explained b

## 1 - País más feliz

Veamos el mínimo y máximo índice de felicidad en los datos

In [ ]:
df21.select(

    min('Ladder_score'),
    max('Ladder_score')

).show()

Veamos el máximo y el mínimo error en los índices de felicidad para asegurarnos que el índice de felicidad no es afectado por el error

In [ ]:
df21.select(

    min('Standard error of ladder score'),
    max('Standard error of ladder score')

).show()

Sabiendo que el mayor índice está por encima de 7, encontramos el país con el mayor índice de felicidad

In [ ]:
df21.select('Country_name', 'Ladder_score').filter(col('Ladder_score') > 7).show()

In [ ]:
import pandas as pd
data = pd.read_csv('world-happiness-report-2021.csv')
df = pd.DataFrame(data)
df[df['Ladder score'] == df['Ladder score'].max()]

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.842,0.032,7.904,7.78,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253


verificamos el error de todos los países con un índice de felicidad mayor a 7 para asegurarnos que el índice de felicidad no es afectado

In [ ]:
df21.select('Country_name', 'Ladder_score', 'Standard error of ladder score').where(col('Ladder score') >= 7).show()

+--------------+------------+------------------------------+
|  Country_name|Ladder_score|Standard error of ladder score|
+--------------+------------+------------------------------+
|       Finland|       7.842|                         0.032|
|       Denmark|        7.62|                         0.035|
|   Switzerland|       7.571|                         0.036|
|       Iceland|       7.554|                         0.059|
|   Netherlands|       7.464|                         0.027|
|        Norway|       7.392|                         0.035|
|        Sweden|       7.363|                         0.036|
|    Luxembourg|       7.324|                         0.037|
|   New Zealand|       7.277|                         0.040|
|       Austria|       7.268|                         0.036|
|     Australia|       7.183|                         0.041|
|        Israel|       7.157|                         0.034|
|       Germany|       7.155|                         0.040|
|        Canada|       7

Por lo tanto, podemos concluir que **Finlandia** es el país más feliz del mundo, según nuestra data

## 2 - País más feliz por continente

Obtenemos los puntajes más altos por Continente mediante la columna Regional indicator

In [ ]:
df.iloc[df.groupby('Regional indicator').agg(paises_felices = ('Ladder score', lambda df: df.idxmax())).paises_felices]

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
17,Czech Republic,Central and Eastern Europe,6.965,0.049,7.062,6.868,10.556,0.947,70.807,0.858,-0.208,0.868,2.43,1.370,1.090,0.703,0.580,0.052,0.046,3.124
41,Uzbekistan,Commonwealth of Independent States,6.179,0.068,6.312,6.045,8.836,0.918,65.255,0.970,0.311,0.515,2.43,0.769,1.027,0.528,0.716,0.391,0.271,2.477
23,Taiwan Province of China,East Asia,6.584,0.038,6.659,6.510,10.871,0.898,69.600,0.784,-0.070,0.721,2.43,1.480,0.982,0.665,0.490,0.142,0.139,2.687
15,Costa Rica,Latin America and Caribbean,7.069,0.056,7.179,6.960,9.880,0.891,71.400,0.934,-0.126,0.809,2.43,1.134,0.966,0.722,0.673,0.105,0.083,3.387
11,Israel,Middle East and North Africa,7.157,0.034,7.224,7.090,10.575,0.939,73.503,0.800,0.031,0.753,2.43,1.376,1.074,0.788,0.509,0.208,0.119,3.083
8,New Zealand,North America and ANZ,7.277,0.040,7.355,7.198,10.643,0.948,73.400,0.929,0.134,0.242,2.43,1.400,1.094,0.785,0.665,0.276,0.445,2.612
86,Nepal,South Asia,5.269,0.070,5.406,5.132,8.120,0.774,64.233,0.782,0.152,0.727,2.43,0.519,0.702,0.496,0.488,0.287,0.135,2.642
31,Singapore,Southeast Asia,6.377,0.043,6.460,6.293,11.488,0.915,76.953,0.927,-0.018,0.082,2.43,1.695,1.019,0.897,0.664,0.176,0.547,1.379
49,Mauritius,Sub-Saharan Africa,6.049,0.059,6.165,5.933,10.008,0.905,66.701,0.867,-0.054,0.789,2.43,1.178,0.996,0.574,0.590,0.153,0.096,2.462
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.000,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253


## 3 - País que ocupó más veces el primer lugar

Cargamos nuestro data de los años anteriores al 2021

In [ ]:
dfanteriores = spark.read.option('header','true').csv('./world-happiness-report.csv')

verificamos el tipo de dato de las columnas

In [ ]:
dfanteriores.printSchema()

root
 |-- Country name: string (nullable = true)
 |-- year: string (nullable = true)
 |-- Life Ladder: float (nullable = true)
 |-- Log GDP per capita: string (nullable = true)
 |-- Social support: string (nullable = true)
 |-- Healthy life expectancy at birth: string (nullable = true)
 |-- Freedom to make life choices: string (nullable = true)
 |-- Generosity: string (nullable = true)
 |-- Perceptions of corruption: string (nullable = true)
 |-- Positive affect: string (nullable = true)
 |-- Negative affect: string (nullable = true)



Convertimos la columna Life Ladder a decimal, ya que es string

In [ ]:
dfanteriores = dfanteriores.withColumn('Life Ladder', col('Life Ladder').cast('float'))

In [ ]:
dfanteriores.printSchema()

root
 |-- Country name: string (nullable = true)
 |-- year: string (nullable = true)
 |-- Life Ladder: float (nullable = true)
 |-- Log GDP per capita: string (nullable = true)
 |-- Social support: string (nullable = true)
 |-- Healthy life expectancy at birth: string (nullable = true)
 |-- Freedom to make life choices: string (nullable = true)
 |-- Generosity: string (nullable = true)
 |-- Perceptions of corruption: string (nullable = true)
 |-- Positive affect: string (nullable = true)
 |-- Negative affect: string (nullable = true)



Creamos un dataframe para pandas

In [ ]:
datat = pd.read_csv('world-happiness-report.csv')
dft = pd.DataFrame(datat)

Obtenemos el mayor puntaje por cada año

In [ ]:
primeros = dft.iloc[dft.groupby('year').agg(primer_lugar = ('Life Ladder', lambda dft: dft.idxmax())).primer_lugar]

Convertimos a lista la columna de los países con mayor puntaje por años

In [ ]:
lista_primeros = primeros['Country name'].to_list()

Obtenemos las veces que se repiten los países que más aparecen

In [ ]:
print(lista_primeros.count('Denmark'))
print(lista_primeros.count('Finland'))

7
6


Los dos países que más aparecen como países felices en el mundo son: Dinamarca y Finlandia.


Concluimos que el país que más veces ha ocupado el primer lugar dentro de los países más felices del mundo es **Dinamarca**

## 4 - País con mayor GDP y su posición en Felicidad

In [ ]:
dft[dft['Log GDP per capita'] == dft['Log GDP per capita'].max()]

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
1026,Luxembourg,2019,7.404,11.648,0.912,72.6,0.93,-0.045,0.39,0.789,0.212


Luxemburgo es el país con mayor PIB per capita

Países con mayor felicidad ordenados de mayor a menor

In [ ]:
df_ordenado = pd.DataFrame(dft.sort_values(by='Life Ladder', ascending=False))
df_ordenado.reset_index()

,index,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,449,Denmark,2005,8.019,10.851,0.972,69.60,0.971,NaN,0.237,0.860,0.154
1,451,Denmark,2008,7.971,10.880,0.954,70.08,0.970,0.272,0.248,0.757,0.163
2,563,Finland,2020,7.889,10.750,0.962,72.10,0.962,-0.116,0.164,0.744,0.193
3,561,Finland,2018,7.858,10.783,0.962,71.90,0.938,-0.127,0.199,0.782,0.182
4,450,Denmark,2007,7.834,10.891,0.954,69.92,0.932,0.240,0.206,0.828,0.194
...,...,...,...,...,...,...,...,...,...,...,...,...
1944,1947,Zimbabwe,2019,2.694,7.950,0.759,56.20,0.632,-0.064,0.831,0.716,0.235
1945,311,Central African Republic,2016,2.693,6.785,0.290,44.90,0.624,0.033,0.859,0.579,0.494
1946,1667,Syria,2013,2.688,8.396,0.585,58.76,0.455,0.225,0.663,0.387,0.622
1947,9,Afghanistan,2017,2.662,7.697,0.491,52.80,0.427,-0.121,0.954,0.496,0.371


Filtramos solo del año 2020: Países más felices del 2020

In [ ]:
import numpy as np

df_2020 = df_ordenado[df_ordenado.year == 2020]
#Reiniciamos el índice para tener los países más felices desde 1 en adelante
df_2020.reset_index(inplace=True, drop=True)
#como el índice comienza en 0, reseteamos el índice para comenzar en 1
df_2020.index = np.arange(1,len(df_2020)+1)
df_2020

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
1,Finland,2020,7.889,10.750,0.962,72.1,0.962,-0.116,0.164,0.744,0.193
2,Iceland,2020,7.575,10.824,0.983,73.0,0.949,0.160,0.644,0.863,0.172
3,Denmark,2020,7.515,10.910,0.947,73.0,0.938,0.052,0.214,0.818,0.227
4,Switzerland,2020,7.508,11.081,0.946,74.7,0.917,-0.064,0.280,0.769,0.193
5,Netherlands,2020,7.504,10.901,0.944,72.5,0.935,0.151,0.281,0.784,0.247
...,...,...,...,...,...,...,...,...,...,...,...
91,Cambodia,2020,4.377,8.362,0.724,62.4,0.963,0.052,0.863,0.878,0.390
92,India,2020,4.225,8.703,0.617,60.9,0.906,0.075,0.780,0.752,0.383
93,Jordan,2020,4.094,9.150,0.709,67.2,0.779,-0.150,NaN,NaN,NaN
94,Tanzania,2020,3.786,7.881,0.740,58.5,0.830,0.295,0.521,0.686,0.271


Encontramos el país con mayor GDP per capita dentro de los países con mayor felicidad del 2020

In [ ]:
df_2020[df_2020['Log GDP per capita'] == df_2020['Log GDP per capita'].max()]

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
13,Ireland,2020,7.035,11.323,0.96,72.5,0.882,0.014,0.356,0.797,0.246


Por lo tanto, el país con mayor GDP per capita del 2020 se encuentra en el lugar número 13 de los países más felices del mundo en el 2020.

## 5 - Variación de GDP promedio entre 2020 y 2021

Obtener el promedio de GDP del 2020

Convertir la columna GDP a float

In [ ]:
from pyspark.sql.types import IntegerType

df_2020 = spark.read.csv('./world-happiness-report.csv', header=True)
df_2021 = spark.read.csv('./world-happiness-report-2021.csv', header=True)

df_2020 = df_2020.withColumn('Log GDP per capita', col('Log GDP per capita').cast('float'))
df_2021 = df_2021.withColumn('Logged GDP per capita', col('Logged GDP per capita').cast('float'))
df_2020.printSchema()
df_2021.printSchema()

root
 |-- Country name: string (nullable = true)
 |-- year: string (nullable = true)
 |-- Life Ladder: string (nullable = true)
 |-- Log GDP per capita: float (nullable = true)
 |-- Social support: string (nullable = true)
 |-- Healthy life expectancy at birth: string (nullable = true)
 |-- Freedom to make life choices: string (nullable = true)
 |-- Generosity: string (nullable = true)
 |-- Perceptions of corruption: string (nullable = true)
 |-- Positive affect: string (nullable = true)
 |-- Negative affect: string (nullable = true)

root
 |-- Country name: string (nullable = true)
 |-- Regional indicator: string (nullable = true)
 |-- Ladder score: string (nullable = true)
 |-- Standard error of ladder score: string (nullable = true)
 |-- upperwhisker: string (nullable = true)
 |-- lowerwhisker: string (nullable = true)
 |-- Logged GDP per capita: float (nullable = true)
 |-- Social support: string (nullable = true)
 |-- Healthy life expectancy: string (nullable = true)
 |-- Freedom 

Obtención del promedio GDP del 2020

In [ ]:
from pyspark.sql.functions import mean
prom_2020 = df_2020.select(mean('Log GDP per capita')).collect()[0][0]
print(prom_2020)

9.368452701777983


Obtenención del promedio GDP del 2021

In [ ]:
prom_2021 = df_2021.select(mean('Logged GDP per capita')).collect()[0][0]
print(prom_2021)

9.432208032416018


Obteniendo la diferencia entre 2021 y 2020

In [ ]:
variacion = round(prom_2021 - prom_2020,4)
print(f'La variación entre el 2021 y 2020 fue de {variacion}')

La variación entre el 2021 y 2020 fue de 0.0638


Convirtiendo la variación a porcentaje respecto al 2021

In [ ]:
porcentaje_variacion = round((variacion * 100)/ prom_2021,2)
print(f'El porcentaje de varición entre el 2021 y el 2020 fue de: {porcentaje_variacion}')

El porcentaje de varición entre el 2021 y el 2020 fue de: 0.68


## 6 - País con mayor expectativa de vida

Encontrando el valor máximo en la columna de esperanza de vida

In [ ]:
df_2020.select(max('Healthy life expectancy at birth')).collect()[0][0]

'77.100'

Encontrando el país con mayor esperanza de vida en el 2019

In [ ]:
from pyspark.sql.functions import col
pais_mayor_expectativa_vida = df_2020.orderBy(col('Healthy life expectancy at birth').desc()).first()
pais_mayor_expectativa_vida['Country name']

'Singapore'